In [1]:
from selenium import webdriver
import pandas as pd
import time
import requests
from random import randint
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

In [2]:
driver = webdriver.Chrome('C:/Users/abdurrahman.kocukcu/chromedriver/chromedriver.exe')

In [3]:
sub_category_list = ['Men', 'Clothing', 'Shirts', 'Tank Tops']

In [4]:
page = 1
page_limit = 10
search_keyword = 'men'

In [5]:
def open_homepage():
    driver.get('https://www.amazon.com/')
    driver.maximize_window()

In [6]:
def open_camo_search_results():
    menu_item = driver.find_element_by_xpath("//*[@id='searchDropdownBox']/option[@value='search-alias=fashion']")
    menu_item.click()

    search_bar = driver.find_element_by_xpath("//*[@id='twotabsearchtextbox']")
    search_bar.send_keys(search_keyword)

    submit_button = driver.find_element_by_xpath("//*[@value='Go']")
    submit_button.click()

In [7]:
def get_product_urls():

    real_products = []
    product_urls = []

    products = driver.find_elements_by_css_selector("#search > div.sg-row > div.sg-col-20-of-24.sg-col-28-of-32.sg-col-16-of-20.sg-col.s-right-column.sg-col-32-of-36.sg-col-8-of-12.sg-col-12-of-16.sg-col-24-of-28 > div > span:nth-child(5) > div.s-result-list.sg-row > div")
    print("product count : ",len(products))
    for idx, product in enumerate(products):
        try:
            if "AdHolder" not in product.get_attribute("class") and product.get_attribute("data-index") is not None and "search_result_" in product.get_attribute("data-cel-widget"):
                real_products.append(product)
        except Exception as e:
            print(e)

    print("real product count : ",len(real_products))
    #print(real_products[0].get_attribute("data-asin"))

    for real_product in real_products:
        product_anchor = real_product.find_element_by_css_selector('[data-component-type="s-product-image"] > a')
        product_url =  product_anchor.get_attribute("href")
        product_urls.append(product_url)

    return product_urls

In [8]:
def open_product_detail_page(product_url):
    driver.switch_to.window(driver.window_handles[1])
    driver.get(product_url)

In [18]:
def select_swatch_elements(product_number):
    selected_product_number = ''
    swatch_elements = []
    data_defaultasin_list = []
    
    try:
        swatch_elements = driver.find_elements_by_css_selector("ul.imageSwatches > li")
    except Exception as e:
        print(e)

    if len(swatch_elements)>0:
        for idx, swatch_element in enumerate(swatch_elements):
            data_defaultasin =  swatch_element.get_attribute("data-defaultasin")
            data_defaultasin_list.append(data_defaultasin)
            
        for data_default_asin_idx, data_default_asin_element in enumerate(data_defaultasin_list):
            swatch_element_button = driver.find_element_by_xpath('//li[@data-defaultasin="'+data_default_asin_element+'"]/span/div/span/span/span/button')
            random_click_time = randint(2,4)
            swatch_click_action = ActionChains(driver).click(swatch_element_button)
            swatch_click_action.perform()
            time.sleep(random_click_time)
            selected_product_number = str(product_number)+'-'+str(data_default_asin_idx)
            download_product_detail_images(selected_product_number)
        
        
    else:
        selected_product_number = str(product_number)+'-'+'0'
        download_product_detail_images(selected_product_number)

In [10]:
def download_product_detail_images(number):
    hover_elements = driver.find_elements_by_css_selector("li.a-spacing-small.item.imageThumbnail.a-declarative")

    for idx, hover_element in enumerate(hover_elements):
        random_hover_time = randint(2,4)
        hover_action = ActionChains(driver).move_to_element(hover_element)
        hover_action.perform()
        time.sleep(random_hover_time)

    hires_images =  driver.find_elements_by_css_selector("#main-image-container > ul > li.maintain-height > span > span > div > img")


    for idx,hires_image in enumerate(hires_images):
        random_load_time = randint(3,9)
        
        try:
            my_image_url = hires_image.get_attribute("data-old-hires")
            my_image_filename='images/'+str(page)+'-'+number+'-'+str(idx)+'.jpg'
            my_response = requests.get(url=my_image_url, stream=True, timeout=20)
            if my_response.status_code == 200:
                with open(my_image_filename, 'wb') as my_file:
                    my_file.write(my_response.content)
                del my_response
            pass
        except Exception as e:
            print(e)
        
        
        time.sleep(random_load_time)

In [11]:
def go_to_next_page():
    # next_page_button = driver.find_element_by_css_selector("#search > div.sg-row > div.sg-col-20-of-24.sg-col-28-of-32.sg-col-16-of-20.sg-col.s-right-column.sg-col-32-of-36.sg-col-8-of-12.sg-col-12-of-16.sg-col-24-of-28 > div > span:nth-child(9) > div > div > div > ul > li.a-last > a")
    next_page_button = driver.find_element_by_xpath("//ul[@class='a-pagination']/li[@class='a-last']/a[contains(text(), 'Next')]")
    next_page_button.click()

In [12]:
def go_to_sub_category(sub_category):
    nav_links = products = driver.find_elements_by_css_selector("#departments > ul.a-unordered-list.a-nostyle.a-vertical.a-spacing-medium > li")
    for nav_link in nav_links:
        # we check whether this element exists. As SO suggested we should use find_element"S"
        # https://stackoverflow.com/a/36166132
        if nav_link.find_elements_by_css_selector("span > a"):
            nav_link_item = nav_link.find_element_by_css_selector("span > a")
            nav_link_url = nav_link_item.get_attribute("href")
            nav_link_text = nav_link_item.find_element_by_css_selector("span").text
            if nav_link_text == sub_category:
                nav_link_item.click()

In [13]:
open_homepage()

In [14]:
driver.execute_script("window.open('','_blank');")
driver.switch_to.window(driver.window_handles[0])

In [15]:
open_camo_search_results()

In [16]:
for sub_category in sub_category_list:
    random_sub_category_page_time = randint(5,10)
    try:
        go_to_sub_category(sub_category)
        time.sleep(random_sub_category_page_time)
    except Exception as e:
        print(e)

Message: stale element reference: element is not attached to the page document
  (Session info: chrome=78.0.3904.97)

Message: stale element reference: element is not attached to the page document
  (Session info: chrome=78.0.3904.97)

Message: stale element reference: element is not attached to the page document
  (Session info: chrome=78.0.3904.97)

Message: stale element reference: element is not attached to the page document
  (Session info: chrome=78.0.3904.97)



In [20]:
while page<page_limit:
    
    random_next_page_time = randint(5,10)
    
    product_urls = get_product_urls()

    for idx,product_url  in enumerate(product_urls):
        random_open_time = randint(2,4)
        try:
            open_product_detail_page(product_url)
            select_swatch_elements(idx)
            pass
        except Exception as e:
            print(e)
        driver.switch_to.window(driver.window_handles[0])
        time.sleep(random_open_time)

    go_to_next_page()
    time.sleep(random_next_page_time)
    page += 1

product count :  61
real product count :  48
https://www.amazon.com/Amazon-Essentials-Regular-Fit-Ringer-Large/dp/B07HJ7CS73/ref=sr_1_1?dchild=1&keywords=men&qid=1573470336&rnid=2476517011&s=apparel&sr=1-1
https://www.amazon.com/Amazon-Essentials-Slim-Fit-Ringer-Black/dp/B07HJDJ6FC/ref=sr_1_2?dchild=1&keywords=men&qid=1573470336&rnid=2476517011&s=apparel&sr=1-2
https://www.amazon.com/Hurley-Mens-Graphic-PHOTOBLU-White/dp/B075ZV3S94/ref=sr_1_3?dchild=1&keywords=men&qid=1573470336&rnid=2476517011&s=apparel&sr=1-3
https://www.amazon.com/Fruit-Loom-Heavy-Cotton-BLACK/dp/B00I2ZCNZA/ref=sr_1_4?dchild=1&keywords=men&qid=1573470336&rnid=2476517011&s=apparel&sr=1-4
https://www.amazon.com/Hanes-Mens-X-Temp-Black-XX-Large/dp/B01AZS4CJG/ref=sr_1_5?dchild=1&keywords=men&qid=1573470336&rnid=2476517011&s=apparel&sr=1-5
https://www.amazon.com/Polo-Ralph-Lauren-Classic-Ribbed/dp/B00W57U6HI/ref=sr_1_6?dchild=1&keywords=men&qid=1573470336&rnid=2476517011&s=apparel&sr=1-6
https://www.amazon.com/ComfortWas

KeyboardInterrupt: 

In [19]:
driver.switch_to.window(driver.window_handles[0])

In [ ]:
driver.close()